##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial introduces word embeddings. It contains complete code to train word embeddings from scratch on a small dataset, and to visualize these embeddings using the [Embedding Projector](http://projector.tensorflow.org) (shown in the image below).

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## Representing text as numbers

Machine learning models take vectors (arrays of numbers) as input. When working with text, the first thing we must do come up with a strategy to convert strings to numbers (or to "vectorize" the text) before feeding it to the model. In this section, we will look at three strategies for doing so.

### One-hot encodings

As a first idea, we might "one-hot" encode each word in our vocabulary. Consider the sentence "The cat sat on the mat". The vocabulary (or unique words) in this sentence is (cat, mat, on, sat, the). To represent each word, we will create a zero vector with length equal to the vocabulary, then place a one in the index that corresponds to the word. This approach is shown in the following diagram.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagram of one-hot encodings" width="400" />

To create a vector that contains the encoding of the sentence, we could then concatenate the one-hot vectors for each word.

Key point: This approach is inefficient. A one-hot encoded vector is sparse (meaning, most indicices are zero). Imagine we have 10,000 words in the vocabulary. To one-hot encode each word, we would create a vector where 99.99% of the elements are zero.

### Encode each word with a unique number

A second approach we might try is to encode each word using a unique number. Continuing the example above, we could assign 1 to "cat", 2 to "mat", and so on. We could then encode the sentence "The cat sat on the mat" as a dense vector like [5, 1, 4, 3, 5, 2]. This appoach is efficient. Instead of a sparse vector, we now have a dense one (where all elements are full).

There are two downsides to this approach, however:

* The integer-encoding is arbitrary (it does not capture any relationship between words).

* An integer-encoding can be challenging for a model to interpret. A linear classifier, for example, learns a single weight for each feature. Because there is no relationship between the similarity of any two words and the similarity of their encodings, this feature-weight combination is not meaningful.

### Word embeddings

Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding. Importantly, we do not have to specify this encoding by hand. An embedding is a dense vector of floating point values (the length of the vector is a parameter you specify). Instead of specifying the values for the embedding manually, they are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer). It is common to see word embeddings that are 8-dimensional (for small datasets), up to 1024-dimensions when working with large datasets. A higher dimensional embedding can capture fine-grained relationships between words, but takes more data to learn.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

Above is a diagram for a word embedding. Each word is represented as a 4-dimensional vector of floating point values. Another way to think of an embedding is as "lookup table". After these weights have been learned, we can encode each word by looking up the dense vector it corresponds to in the table.

## Setup

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Using the Embedding layer

Keras makes it easy to use word embeddings. Let's take a look at the [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.

The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings). The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.



In [4]:
embedding_layer = layers.Embedding(1000, 5)



When you create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

In [5]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.01616658, -0.03501402, -0.04862527,  0.03040066, -0.04133261],
       [ 0.03420496, -0.03333137,  0.03198669,  0.04705662,  0.00031972],
       [ 0.02208818, -0.0395545 , -0.03491404,  0.01166359, -0.02771395]],
      dtype=float32)

For text or sequence problems, the Embedding layer takes a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of integers. It can embed sequences of variable lengths. You could feed into the embedding layer above batches with shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15).

The returned tensor has one more axis than the input, the embedding vectors are aligned along the new last axis. Pass it a `(2, 3)` input batch and the output is `(2, 3, N)`


In [6]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`. To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. You could use an RNN, Attention, or pooling layer before passing it to a Dense layer. This tutorial uses pooling because it's simplest. The [Text Classification with an RNN](text_classification_rnn.ipynb) tutorial is a good next step.

## Learning embeddings from scratch

In this tutorial you will train a sentiment classifier on IMDB movie reviews. In the process, the model will learn embeddings from scratch. We will use to a preprocessed dataset.

To load a text dataset from scratch see the  [Loading text tutorial](../load_data/text.ipynb).

In [7]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Get the encoder (`tfds.features.text.SubwordTextEncoder`), and have a quick look at the vocabulary. 

The "\_" in the vocabulary represent spaces. Note how the vocabulary includes whole words (ending with "\_") and partial words which it can use to build larger words:

In [8]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

Movie reviews can be different lengths. We will use the `padded_batch` method to standardize the lengths of the reviews.

In [9]:
padded_shapes = ([None],())
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

As imported, the text of reviews is integer-encoded (each integer represents a specific word or word-part in the vocabulary).

Note the trailing zeros, because the batch is padded to the longest example.

In [10]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()
train_labels.numpy()

array([1, 0, 0, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

### Create a simple model

We will use the [Keras Sequential API](../../guide/keras) to define our model. In this case it is a "Continuous bag of words" style model.

* Next the Embedding layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch, sequence, embedding)`.

* Next, a GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.

* This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.

* The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability (or confidence level) that the review is positive.

Caution: This model doesn't use masking, so the zero-padding is used as part of the input, so the padding length may affect the output.  To fix this, see the [masking and padding guide](../../guide/keras/masking_and_padding).

In [11]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


### Compile and train the model

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10


    173/Unknown - 2s 2s/step - loss: 0.6921 - accuracy: 0.60 - 2s 1s/step - loss: 0.6934 - accuracy: 0.55 - 2s 736ms/step - loss: 0.6938 - accuracy: 0.500 - 2s 560ms/step - loss: 0.6948 - accuracy: 0.425 - 2s 453ms/step - loss: 0.6947 - accuracy: 0.420 - 2s 382ms/step - loss: 0.6947 - accuracy: 0.416 - 2s 333ms/step - loss: 0.6944 - accuracy: 0.442 - 2s 294ms/step - loss: 0.6942 - accuracy: 0.450 - 2s 263ms/step - loss: 0.6942 - accuracy: 0.444 - 2s 239ms/step - loss: 0.6939 - accuracy: 0.460 - 2s 226ms/step - loss: 0.6937 - accuracy: 0.472 - 3s 211ms/step - loss: 0.6936 - accuracy: 0.483 - 3s 196ms/step - loss: 0.6937 - accuracy: 0.476 - 3s 183ms/step - loss: 0.6934 - accuracy: 0.492 - 3s 172ms/step - loss: 0.6932 - accuracy: 0.500 - 3s 163ms/step - loss: 0.6935 - accuracy: 0.493 - 3s 155ms/step - loss: 0.6939 - accuracy: 0.482 - 3s 150ms/step - loss: 0.6939 - accuracy: 0.477 - 3s 144ms/step - loss: 0.6937 - accuracy: 0.489 - 3s 137ms/step - loss: 0.6935 - accuracy: 0.495 - 3s 132ms/s

    345/Unknown - 6s 37ms/step - loss: 0.6933 - accuracy: 0.50 - 6s 37ms/step - loss: 0.6933 - accuracy: 0.50 - 6s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6933 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6933 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6933 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6933 - accuracy: 0.49 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 37ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.50 - 7s 36ms/step - loss: 0.6932 - accuracy: 0.

    512/Unknown - 11s 33ms/step - loss: 0.6926 - accuracy: 0.514 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.515 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.515 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.515 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.516 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.516 - 11s 33ms/step - loss: 0.6926 - accuracy: 0.517 - 12s 33ms/step - loss: 0.6926 - accuracy: 0.518 - 12s 33ms/step - loss: 0.6926 - accuracy: 0.518 - 12s 33ms/step - loss: 0.6926 - accuracy: 0.519 - 12s 33ms/step - loss: 0.6926 - accuracy: 0.519 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.519 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.521 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.521 - 12s 32ms/step - loss: 0.6926 - accuracy: 0.520 - 12

    679/Unknown - 15s 30ms/step - loss: 0.6912 - accuracy: 0.555 - 15s 30ms/step - loss: 0.6912 - accuracy: 0.555 - 15s 30ms/step - loss: 0.6911 - accuracy: 0.555 - 15s 30ms/step - loss: 0.6911 - accuracy: 0.556 - 15s 30ms/step - loss: 0.6911 - accuracy: 0.556 - 15s 30ms/step - loss: 0.6911 - accuracy: 0.555 - 16s 30ms/step - loss: 0.6911 - accuracy: 0.556 - 16s 30ms/step - loss: 0.6911 - accuracy: 0.556 - 16s 30ms/step - loss: 0.6911 - accuracy: 0.557 - 16s 30ms/step - loss: 0.6911 - accuracy: 0.557 - 16s 30ms/step - loss: 0.6911 - accuracy: 0.557 - 16s 30ms/step - loss: 0.6910 - accuracy: 0.557 - 16s 30ms/step - loss: 0.6910 - accuracy: 0.558 - 16s 30ms/step - loss: 0.6910 - accuracy: 0.558 - 16s 30ms/step - loss: 0.6910 - accuracy: 0.559 - 16s 30ms/step - loss: 0.6909 - accuracy: 0.559 - 16s 30ms/step - loss: 0.6909 - accuracy: 0.559 - 16s 30ms/step - loss: 0.6909 - accuracy: 0.559 - 16s 30ms/step - loss: 0.6909 - accuracy: 0.559 - 16s 30ms/step - loss: 0.6909 - accuracy: 0.559 - 16

...

2167/2500 [=========================>....] - ETA: 13s - loss: 0.1268 - accuracy: 0.956 - ETA: 13s - loss: 0.1264 - accuracy: 0.956 - ETA: 13s - loss: 0.1266 - accuracy: 0.956 - ETA: 13s - loss: 0.1265 - accuracy: 0.956 - ETA: 13s - loss: 0.1262 - accuracy: 0.957 - ETA: 13s - loss: 0.1270 - accuracy: 0.956 - ETA: 13s - loss: 0.1267 - accuracy: 0.957 - ETA: 13s - loss: 0.1265 - accuracy: 0.957 - ETA: 13s - loss: 0.1262 - accuracy: 0.957 - ETA: 13s - loss: 0.1262 - accuracy: 0.957 - ETA: 13s - loss: 0.1264 - accuracy: 0.956 - ETA: 13s - loss: 0.1264 - accuracy: 0.956 - ETA: 12s - loss: 0.1260 - accuracy: 0.956 - ETA: 12s - loss: 0.1267 - accuracy: 0.956 - ETA: 12s - loss: 0.1265 - accuracy: 0.956 - ETA: 12s - loss: 0.1263 - accuracy: 0.956 - ETA: 12s - loss: 0.1259 - accuracy: 0.956 - ETA: 12s - loss: 0.1265 - accuracy: 0.956 - ETA: 12s - loss: 0.1262 - accuracy: 0.956 - ETA: 12s - loss: 0.1269 - accuracy: 0.956 - ETA: 12s - loss: 0.1269 - accuracy: 0.956 - ETA: 12s - loss: 0.1267 - accur

2500/2500 [==============================] - ETA: 3s - loss: 0.1248 - accuracy: 0.95 - ETA: 3s - loss: 0.1247 - accuracy: 0.95 - ETA: 3s - loss: 0.1248 - accuracy: 0.95 - ETA: 3s - loss: 0.1247 - accuracy: 0.95 - ETA: 3s - loss: 0.1247 - accuracy: 0.95 - ETA: 3s - loss: 0.1249 - accuracy: 0.95 - ETA: 3s - loss: 0.1246 - accuracy: 0.95 - ETA: 2s - loss: 0.1251 - accuracy: 0.95 - ETA: 2s - loss: 0.1256 - accuracy: 0.95 - ETA: 2s - loss: 0.1257 - accuracy: 0.95 - ETA: 2s - loss: 0.1255 - accuracy: 0.95 - ETA: 2s - loss: 0.1255 - accuracy: 0.95 - ETA: 2s - loss: 0.1253 - accuracy: 0.95 - ETA: 2s - loss: 0.1252 - accuracy: 0.95 - ETA: 2s - loss: 0.1250 - accuracy: 0.95 - ETA: 2s - loss: 0.1248 - accuracy: 0.95 - ETA: 2s - loss: 0.1248 - accuracy: 0.95 - ETA: 2s - loss: 0.1247 - accuracy: 0.95 - ETA: 2s - loss: 0.1249 - accuracy: 0.95 - ETA: 1s - loss: 0.1248 - accuracy: 0.95 - ETA: 1s - loss: 0.1248 - accuracy: 0.95 - ETA: 1s - loss: 0.1248 - accuracy: 0.95 - ETA: 1s - loss: 0.1248 - accura

1099/2500 [============>.................] - ETA: 12:38 - loss: 0.4672 - accuracy: 0.800 - ETA: 2:28 - loss: 0.2754 - accuracy: 0.916 - ETA: 1:25 - loss: 0.2135 - accuracy: 0.93 - ETA: 1:07 - loss: 0.2165 - accuracy: 0.92 - ETA: 55s - loss: 0.1920 - accuracy: 0.9348 - ETA: 50s - loss: 0.1899 - accuracy: 0.932 - ETA: 54s - loss: 0.1786 - accuracy: 0.936 - ETA: 55s - loss: 0.1688 - accuracy: 0.940 - ETA: 49s - loss: 0.1528 - accuracy: 0.948 - ETA: 44s - loss: 0.1460 - accuracy: 0.950 - ETA: 40s - loss: 0.1502 - accuracy: 0.950 - ETA: 37s - loss: 0.1410 - accuracy: 0.955 - ETA: 36s - loss: 0.1410 - accuracy: 0.953 - ETA: 35s - loss: 0.1342 - accuracy: 0.955 - ETA: 33s - loss: 0.1324 - accuracy: 0.957 - ETA: 32s - loss: 0.1294 - accuracy: 0.960 - ETA: 31s - loss: 0.1351 - accuracy: 0.958 - ETA: 30s - loss: 0.1281 - accuracy: 0.961 - ETA: 29s - loss: 0.1247 - accuracy: 0.962 - ETA: 28s - loss: 0.1310 - accuracy: 0.958 - ETA: 28s - loss: 0.1328 - accuracy: 0.957 - ETA: 27s - loss: 0.1334 - a

2212/2500 [=========================>....] - ETA: 13s - loss: 0.1161 - accuracy: 0.962 - ETA: 13s - loss: 0.1163 - accuracy: 0.962 - ETA: 13s - loss: 0.1159 - accuracy: 0.962 - ETA: 13s - loss: 0.1160 - accuracy: 0.962 - ETA: 13s - loss: 0.1160 - accuracy: 0.962 - ETA: 13s - loss: 0.1159 - accuracy: 0.962 - ETA: 13s - loss: 0.1158 - accuracy: 0.962 - ETA: 13s - loss: 0.1161 - accuracy: 0.962 - ETA: 12s - loss: 0.1164 - accuracy: 0.962 - ETA: 12s - loss: 0.1161 - accuracy: 0.962 - ETA: 12s - loss: 0.1161 - accuracy: 0.962 - ETA: 12s - loss: 0.1160 - accuracy: 0.962 - ETA: 12s - loss: 0.1157 - accuracy: 0.962 - ETA: 12s - loss: 0.1156 - accuracy: 0.962 - ETA: 12s - loss: 0.1159 - accuracy: 0.962 - ETA: 12s - loss: 0.1159 - accuracy: 0.962 - ETA: 12s - loss: 0.1158 - accuracy: 0.962 - ETA: 12s - loss: 0.1158 - accuracy: 0.962 - ETA: 12s - loss: 0.1156 - accuracy: 0.962 - ETA: 12s - loss: 0.1162 - accuracy: 0.962 - ETA: 12s - loss: 0.1173 - accuracy: 0.961 - ETA: 12s - loss: 0.1174 - accur

2500/2500 [==============================] - ETA: 2s - loss: 0.1134 - accuracy: 0.96 - ETA: 2s - loss: 0.1132 - accuracy: 0.96 - ETA: 2s - loss: 0.1133 - accuracy: 0.96 - ETA: 2s - loss: 0.1132 - accuracy: 0.96 - ETA: 2s - loss: 0.1133 - accuracy: 0.96 - ETA: 2s - loss: 0.1131 - accuracy: 0.96 - ETA: 2s - loss: 0.1131 - accuracy: 0.96 - ETA: 2s - loss: 0.1132 - accuracy: 0.96 - ETA: 2s - loss: 0.1130 - accuracy: 0.96 - ETA: 2s - loss: 0.1129 - accuracy: 0.96 - ETA: 2s - loss: 0.1128 - accuracy: 0.96 - ETA: 2s - loss: 0.1127 - accuracy: 0.96 - ETA: 1s - loss: 0.1129 - accuracy: 0.96 - ETA: 1s - loss: 0.1127 - accuracy: 0.96 - ETA: 1s - loss: 0.1126 - accuracy: 0.96 - ETA: 1s - loss: 0.1127 - accuracy: 0.96 - ETA: 1s - loss: 0.1126 - accuracy: 0.96 - ETA: 1s - loss: 0.1127 - accuracy: 0.96 - ETA: 1s - loss: 0.1126 - accuracy: 0.96 - ETA: 1s - loss: 0.1126 - accuracy: 0.96 - ETA: 1s - loss: 0.1125 - accuracy: 0.96 - ETA: 1s - loss: 0.1124 - accuracy: 0.96 - ETA: 1s - loss: 0.1122 - accura

With this approach our model reaches a validation accuracy of around 88% (note the model is overfitting, training accuracy is significantly higher).

In [13]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

<Figure size 1200x900 with 1 Axes>

<Figure size 1200x900 with 1 Axes>

## Retrieve the learned embeddings

Next, let's retrieve the word embeddings learned during training. This will be a matrix of shape `(vocab_size, embedding-dimension)`.

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(8185, 16)


We will now write the weights to disk. To use the [Embedding Projector](http://projector.tensorflow.org), we will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).

In [15]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in [Colaboratory](https://colab.research.google.com), you can use the following snippet to download these files to your local machine (or use the file browser, *View -> Table of contents -> File browser*).

In [16]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## Visualize the embeddings

To visualize our embeddings we will upload them to the embedding projector.

Open the [Embedding Projector](http://projector.tensorflow.org/) (this can also run in a local TensorBoard instance).

* Click on "Load data".

* Upload the two files we created above: `vecs.tsv` and `meta.tsv`.

The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful". 

Note: your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.

Note: experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the `Dense(16)` layer, retraining the model, and visualizing the embeddings again.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>


## Next steps


This tutorial has shown you how to train and visualize word embeddings from scratch on a small dataset.

* To learn about recurrent networks see the [Keras RNN Guide](../../guide/keras/rnn.ipynb).

* To learn more about text classification (including the overall workflow, and if you're curious about when to use embeddings vs one-hot encodings) we recommend this practical text classification [guide](https://developers.google.com/machine-learning/guides/text-classification/step-2-5).